# Mapping Receptor Features to Brain Regions

This notebook demonstrates the process of mapping receptor density features to various brain regions using the siibra library. The workflow includes importing necessary libraries, rendering an ASCII tree of features, fetching receptor features linked to cytoarchitectonic regions, retrieving parcellation maps, assigning regions, visualizing component maps, mapping features to URLs, plotting assignments, and generating tabular outputs for further analysis.

In [ ]:
import siibra
import pandas as pd
from tqdm import tqdm
from nilearn import plotting

In [ ]:
siibra.features.render_ascii_tree("Feature")

In [ ]:
# fetch receptor features linked to cytoarchitectonic regions
receptor_features = siibra.features.get(
    siibra.parcellations.get('julich 3.1'), 
    siibra.features.molecular.ReceptorDensityFingerprint
)
regions_with_receptor_features = set()
for feature in receptor_features:
	for region in feature.anchor.regions:
		regions_with_receptor_features.add(region.name)

print("Regions with receptor features:\n - " + "\n - ".join(regions_with_receptor_features))

In [ ]:
# fetch Desikan/Kiliany map in MNI space
desikan_killiany = siibra.parcellations.get('desikan killiany 2006')
mni152 = siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC

dkmap = siibra.get_map(
    parcellation=desikan_killiany,
    space=mni152,
    maptype="labelled"
)

In [ ]:
# fetch Juelich-Brain cytoarchitectonic map in MNI space
julich = siibra.parcellations.get('julich 3.1')
julichmap = siibra.get_map(
	parcellation='julich 3.1',
	space=mni152,
	maptype="statistical"
)

In [ ]:
# Assign Julich-Brain regions to Desikan-Killiany regions and fetch component maps
# TODO: this is taking too long, test with a single region first
assignments = {}
component_maps = {}

for index, dk_region in tqdm(
	enumerate(dkmap.regions), total=1, unit="feature"
):
	with siibra.QUIET:
		component_map = dkmap.fetch(region=dk_region)
		volume = siibra.volumes.volume.from_nifti(component_map, dkmap.space, dk_region)
		assignments[dk_region] = julichmap.assign(volume)
		component_maps[dk_region] = component_map

In [ ]:
# Plot the component map of the first Desikan-Killiany region
plotting.view_img(
    component_maps[dkmap.regions[0]],
    symmetric_cmap=False,
    cmap="magma"
)

### NEXT cells are just for reference, they are not working with the current version of siibra

In [ ]:
# Define the condition for significant matches
condition = 'correlation > 0.3 or contained > 0.8 or contains > 0.8'

# Initialize the dictionary to store features mapped to URLs
features_mapped = {}

# Iterate over receptor features
for f in tqdm(
	receptor_features, total=len(receptor_features), unit='features'
):
	# Construct the URL for the receptor dataset
	url = f"https://search.kg.ebrains.eu/instances/Dataset/{f.id.split('/')[-1]}"
	features_mapped[url] = []
	
	# Access the anatomical regions linked to the receptor density feature
	for juregion in f.anchor.regions:
		# Get the regional map for the specified space and type
		pmap = juregion.get_regional_mask(space=mni152, maptype="labelled")
		
		# Assign the regional map to the DK atlas and store the scores
		assigned = pd.DataFrame([
			{'JuRegion': juregion, 'DkRegion': dkregion, **scores}
			for dkregion, _, scores in dkmap.assign(pmap.fetch())
		])

		# Filter assignments based on the defined condition
		for match in assigned.query(condition).itertuples():
			features_mapped[url].append(match._asdict())
			features_mapped[url][-1].pop('Index')

In [ ]:
# plot the assignments
from nilearn import plotting

features_mapped = {}
for feature_url, assignments in features_mapped.items():
    for assignment in assignments:
        dkmask = dkmap.fetch_regionmap(assignment['DkRegion'])
        pmap = assignment['JuRegion'].get_regional_map(mni152, "continuous")
        view = plotting.plot_stat_map(
            pmap.fetch(), 
            title=f"{assignment['DkRegion'].name} / {assignment['JuRegion'].name} (rho={assignment['correlation']:.1f})"
        )
        view.add_contours(dkmask)

In [ ]:
# create tabular output
out = pd.DataFrame(
    [
        (url, m['JuRegion'].name, m['DkRegion'], m['correlation'], m['overlap'], m['contained'], m['contains'], ) 
        for url, matches in features_mapped.items() for m in matches
    ], 
    columns=['dataset', 'Julich-Brain region', 'DK region', 'correlation', 'overlap', 'containedness (J>DK)', 'containedness (DK>J)']
)
out.to_excel('receptor_datasets_mapped_to_dk.xlsx')